In [1]:
import os
%pwd

'c:\\Users\\LENOVO\\Downloads\\wine_quality\\End_to_End_Wine_Quality\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\LENOVO\\Downloads\\wine_quality\\End_to_End_Wine_Quality'

In [4]:
import pandas as pd
data=pd.read_csv(r'C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\artifacts\data_ingestion\winequality-red.csv')

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [7]:
from Wine_Quality.constants import *
from Wine_Quality.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Wine_Quality.utils.common import logger

In [10]:
class DataTransformation:
    def __init__(self, config):
        self.config = config

    def train_test_spliting(self):
        # Load the dataset
        data = pd.read_csv(self.config.data_path)
        
        # Separate features (X) and target variable (y)
        X = data.drop(columns=['quality'])  # 'quality' is the target variable
        y = data['quality']

        # Feature Selection: Remove highly correlated features
        correlation_matrix = X.corr()
        high_corr_features = set()

        for i in range(len(correlation_matrix.columns)):
            for j in range(i):
                if abs(correlation_matrix.iloc[i, j]) > 0.9:  # Correlation threshold
                    colname = correlation_matrix.columns[i]
                    high_corr_features.add(colname)

        # Drop highly correlated features
        X = X.drop(columns=high_corr_features)

        # Scale the features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Split the data into training and test sets (0.8, 0.2 split, stratified)
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42, stratify=y
        )

        # Save the splits to CSV files
        train = pd.DataFrame(X_train, columns=X.columns)
        train['quality'] = y_train.reset_index(drop=True)

        test = pd.DataFrame(X_test, columns=X.columns)
        test['quality'] = y_test.reset_index(drop=True)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into training and test sets")
        logger.info(f"Training data shape: {train.shape}")
        logger.info(f"Test data shape: {test.shape}")

        print(f"Training data shape: {train.shape}")
        print(f"Test data shape: {test.shape}")


In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-12-28 14:10:52,793: INFO: common: yaml file: C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\config\config.yaml loaded successfully]
[2024-12-28 14:10:52,799: INFO: common: yaml file: C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\params.yaml loaded successfully]
[2024-12-28 14:10:52,800: INFO: common: yaml file: C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\schema.yaml loaded successfully]
[2024-12-28 14:10:52,803: INFO: common: created directory at: artifacts]
[2024-12-28 14:10:52,805: INFO: common: created directory at: artifacts/data_transformation]
[2024-12-28 14:10:52,863: INFO: 2274939760: Split data into training and test sets]
[2024-12-28 14:10:52,863: INFO: 2274939760: Training data shape: (1279, 12)]
[2024-12-28 14:10:52,863: INFO: 2274939760: Test data shape: (320, 12)]
Training data shape: (1279, 12)
Test data shape: (320, 12)
